In [1]:
import pandas as pd

In [2]:
# Load CSV
df = pd.read_csv("/Users/mithilrsh/Desktop/Zidioproject3/Final_Augmented_dataset_Diseases_and_Symptoms.csv")

# View the first few rows
print(df.head())

         diseases  anxiety and nervousness  depression  shortness of breath  \
0  panic disorder                        1           0                    1   
1  panic disorder                        0           0                    1   
2  panic disorder                        1           1                    1   
3  panic disorder                        1           0                    0   
4  panic disorder                        1           1                    0   

   depressive or psychotic symptoms  sharp chest pain  dizziness  insomnia  \
0                                 1                 0          0         0   
1                                 1                 0          1         1   
2                                 1                 0          1         1   
3                                 1                 0          1         1   
4                                 0                 0          0         1   

   abnormal involuntary movements  chest tightness  ... 

In [3]:
# Check for missing values
print(df.isnull().sum())

diseases                            0
anxiety and nervousness             0
depression                          0
shortness of breath                 0
depressive or psychotic symptoms    0
                                   ..
hip weakness                        0
back swelling                       0
ankle stiffness or tightness        0
ankle weakness                      0
neck weakness                       0
Length: 378, dtype: int64


In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["diseases"] = le.fit_transform(df["diseases"])


In [5]:
X = df.drop("diseases", axis=1)
y = df["diseases"]


In [6]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Train the model using random forest and find accuracy

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Use fewer trees to reduce RAM usage
model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.23025370021664743


Train for better accuracy this accuracy is not enough

In [8]:
# Naive Bayes (great for binary inputs)
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()

In [9]:
# Logistic Regression (multi-class)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)


In [10]:
# Decision Tree with depth limit
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=20)


In [11]:
from sklearn.feature_selection import SelectKBest, chi2

# Select top 100 symptoms
selector = SelectKBest(score_func=chi2, k=100)
X_new = selector.fit_transform(X, y)

# Use this new X_new in place of X
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2)


In [12]:
print(df["diseases"].value_counts())


diseases
165    1219
766    1218
481    1218
138    1217
669    1216
       ... 
728       1
626       1
502       1
340       1
487       1
Name: count, Length: 773, dtype: int64


In [13]:
# Filter out diseases with fewer than 50 samples
disease_counts = df["diseases"].value_counts()
common_diseases = disease_counts[disease_counts >= 50].index

df_filtered = df[df["diseases"].isin(common_diseases)]
print("Remaining diseases:", df_filtered["diseases"].nunique())


Remaining diseases: 527


In [14]:
X = df_filtered.drop("diseases", axis=1)
y = df_filtered["diseases"]


In [15]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

print("New shape:", X_resampled.shape, y_resampled.shape)


New shape: (642413, 377) (642413,)


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9189386922783559
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       236
           1       0.98      0.98      0.98       245
           2       0.93      0.96      0.94       258
           4       0.96      0.94      0.95       230
           7       0.90      0.96      0.93       282
           8       0.82      0.87      0.85       247
           9       0.83      0.71      0.77       270
          10       0.94      0.90      0.92       256
          11       0.76      0.67      0.71       248
          12       0.73      0.72      0.72       245
          14       0.70      0.71      0.70       266
          15       0.98      0.95      0.96       229
          16       0.81      0.88      0.85       229
          17       0.90      0.90      0.90       233
          18       0.93      0.91      0.92       251
          19       0.85      0.86      0.85       242
          20       0.95      0.93      0.94       26

In [17]:
import pickle
from sklearn.ensemble import RandomForestClassifier

# Create a tiny model for testing
small_model = RandomForestClassifier(n_estimators=10)
small_model.fit(X_train[:100], y_train[:100])  # just a slice

# Try saving this
with open("test_model.pkl", "wb") as f:
    pickle.dump(small_model, f)


In [5]:
# If your kernel is crashing due to large file size or memory usage, try these strategies:

# 1. Load only a sample of the data for development/testing:
import pandas as pd

# Load only the first N rows (e.g., 5000) to reduce memory usage
df_sample = pd.read_csv("/Users/mithilrsh/Desktop/Zidioproject3/Final_Augmented_dataset_Diseases_and_Symptoms.csv", nrows=5000)

# 2. Drop unnecessary columns early to save memory
# For example, if there are columns you don't need:
# df_sample = df_sample.drop(columns=["unnecessary_column1", "unnecessary_column2"])

# 3. Use a smaller model for initial testing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder  
from sklearn.model_selection import train_test_split
import joblib

le = LabelEncoder()
df_sample["label"] = le.fit_transform(df_sample["diseases"])

X = df_sample.drop(columns=["diseases", "label"])
y = df_sample["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use fewer trees to reduce memory and training time
model = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)
model.fit(X_train, y_train)

joblib.dump(model, "disease_model_small.joblib")
joblib.dump(le, "label_encoder_small.joblib")

print("✅ Small model and label encoder saved successfully.")

# 4. When ready, you can increase nrows or use the full dataset on a machine with more memory.


✅ Small model and label encoder saved successfully.


In [3]:
import joblib

# ✅ Save the feature (symptom) list in order
all_symptoms = X.columns.tolist()
joblib.dump(all_symptoms, "symptoms_list.joblib")

print("✅ Saved symptoms_list.joblib with", len(all_symptoms), "symptoms.")


✅ Saved symptoms_list.joblib with 377 symptoms.


In [6]:
model = joblib.load("disease_model_small.joblib")
label_encoder = joblib.load("label_encoder_small.joblib")


In [7]:
model = joblib.load("disease_model_small.joblib")
label_encoder = joblib.load("label_encoder_small.joblib")
